# Mask R-CNN Tabletop Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [2]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Import the tabletop dataset config
import humanoids_pouring.tabletop_bottles as tabletop

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
TRAINED_MODEL_PATH = os.path.join(MODEL_DIR, "ycb_video_training20190617T2000", "mask_rcnn_ycb_video_training_0010.h5")
assert os.path.exists(TRAINED_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configurations

In [3]:
#config = tabletop.TabletopConfigInference()
config = tabletop.YCBVideoConfigInference()
config.DETECTION_MIN_CONFIDENCE = 0.5
config.display()

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = config.GPU_ID

AttributeError: module 'humanoids_pouring.tabletop_bottles' has no attribute 'YCBVideoConfigInference'

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(TRAINED_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.

In [ ]:
dataset_root_dir = os.path.join(ROOT_DIR, "datasets", "tabletop_20K_YCBVideoClasses")
dataset = tabletop.TabletopDataset()

In [ ]:
dataset_root_dir = os.path.join(ROOT_DIR, "datasets", "YCB_Video_Dataset")
dataset = tabletop.YCBVideoDataset()

In [ ]:
dataset_root_dir = os.path.join(ROOT_DIR, "datasets", "bottles")
dataset = tabletop.YCBVideoDataset()

In [ ]:
dataset.load_class_names(dataset_root_dir)

### Run Object Detection

In [ ]:
# Load dataset
dataset_val = tabletop.YCBVideoDataset()
dataset_val.load_dataset(dataset_root_dir, 'val')

In [ ]:
# Load a random image from val set
dataset_val = tabletop.YCBVideoDataset()
dataset_val.load_dataset(dataset_root_dir, 'val')
random_val_img_info = dataset_val.image_info[random.randint(0, len(dataset_val.image_info))]
image = skimage.io.imread(random_val_img_info['path'])



In [ ]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
image = skimage.io.imread(os.path.join(IMAGE_DIR, '000001-color.png'))

In [ ]:
# Load a random image from val set
random_val_img_info = dataset_val.image_info[random.randint(0, len(dataset_val.image_info))]
image = skimage.io.imread(random_val_img_info['path'])

In [ ]:
# Run detection

results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'])